In [8]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import cv2
from easyocr import Reader
import re


In [9]:
model2 = torch.hub.load('ultralytics/yolov5', 'custom', path='yolov5/runs/train/exp6/weights/best.pt').to('cpu')

Using cache found in C:\Users\kba13/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-4-2 Python-3.11.8 torch-2.2.2 CUDA:0 (NVIDIA GeForce RTX 2070, 8192MiB)

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


In [10]:
reader = Reader(['en'])

In [11]:
class Car:
    def __init__(self, car_coords, plate_coords):
        self.car_coords = car_coords
        self.plate_coords = plate_coords
        self.plate_text = ''

    def __repr__(self):
        return f"Car(car_coords={self.car_coords}, plate_coords={self.plate_coords})"

In [12]:
def readFrame(result):
    labels = np.squeeze(result.xyxy)
    cars = []
    license_plates = []

    for label in labels:
        x1, y1, x2, y2, pred, class_id = label
        if class_id == 1:
            cars.append(label)
        else: 
            license_plates.append(label)
    return cars, license_plates
    

In [13]:
def findLicensePlate(cars, license_plates, frame):
    matched_cars = []
    
    for plate in license_plates:
        x1, y1, x2, y2, pred, class_id = plate
        for car in cars:
            x1car, y1car, x2car, y2car, p, c = car
            if x1car < x1 and x2car > x2 and y1car < y1 and y2car > y2:
                matched_cars.append(Car((int(x1car), int(y1car), int(x2car), int(y2car)), (int(x1), int(y1), int(x2), int(y2))))
                break
            
    for matched_car in matched_cars:
        x1, y1,x2, y2 = matched_car.plate_coords
        rgb_label = frame[y1:y2, x1:x2]
        gray_label = cv2.cvtColor(rgb_label, cv2.COLOR_BGR2GRAY)
        label_result = reader.readtext(gray_label)
        license_plate_text = ''
        
        for bbox, text, prob in label_result:
            filtered_text = re.sub('[^a-zA-Z0-9]', '', text).upper()
            license_plate_text += filtered_text
        
        matched_car.plate_text = license_plate_text 
        
    return matched_cars


In [14]:
cap = cv2.VideoCapture('traffic.mov')
 
while cap.isOpened():
    ret, frame = cap.read()
    
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break
    
    cars = []
    license_plates = []
    matched_cars = []
    
    result = model2(frame)
    
    cars, license_plates = readFrame(result)
    matched_cars = findLicensePlate(cars, license_plates, frame)
    
    for car in matched_cars:
        x1_car, y1_car, x2_car, y2_car = car.car_coords
        top_left_corner = (x1_car, y2_car)
        bottom_right_corner = (x2_car, y1_car)

        cv2.rectangle(frame, top_left_corner, bottom_right_corner, (0, 255, 0), 3)
        cv2.putText(frame, car.plate_text, (x1_car, y1_car-5), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 0, 255), 3)
 
    cv2.imshow('frame', frame)
    
    if cv2.waitKey(1) == ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()
